In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from config import ModelConfig
from model import get_model, DataGenerator
from simple_loader import get_data, dataloader
import numpy as np

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080, compute capability 8.6


## 0. Model Settings

In [ ]:
np.random.seed(ModelConfig.seed)
n_days, n_features = 32, 22
input_shape = (n_days, n_features)

In [ ]:
model = get_model(input_shape=input_shape)

found best model: ./models\AcademyProject-epoch-016+val_loss-0.0065577473.hdf5


In [ ]:
model.summary()

Model: "finance"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
finance_data (InputLayer)       [(None, 32, 22)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 32, 128)      8576        finance_data[0][0]               
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 128)      512         conv1d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 32, 128)      0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [ ]:
print(ModelConfig.epochs, ModelConfig.ES_PATIENCE, ModelConfig.RP_PATIENCE)

100 150 100


In [ ]:
# model.optimizer.learning_rate.assign(5e-4)

## 1. Load finance data from preloading_data.ipynd

In [ ]:
from simple_loader import dataloader
x_dataset, y_dataset = dataloader('./pickle', 'finance_scaled', ['x_dataset_series', 'y_dataset_series'])
x_dataset.shape, y_dataset.shape

((5996058, 32, 22), (5996058, 32))

In [ ]:
def shuffle_arrays(*arrays):
    rng_state = np.random.get_state()
    for array in arrays:
        np.random.set_state(rng_state)
        np.random.shuffle(array)

In [ ]:
# shuffle_arrays(x_dataset, y_dataset)

In [ ]:
validation_split = ModelConfig.val_split
val_len = int(validation_split * x_dataset.shape[0])

val_gen = DataGenerator(x_dataset[-val_len:], y_dataset[-val_len:], ModelConfig.batch_size)
train_gen = DataGenerator(x_dataset[:-val_len], y_dataset[:-val_len], ModelConfig.batch_size, shuffle=True)

## 3. Fitting Model

In [ ]:
history = model.fit(
    train_gen,
    epochs=ModelConfig.epochs,
    validation_data=val_gen,
    verbose=1, shuffle=True,
    callbacks=ModelConfig.callbacks,
)

1172/1172 [==============================] - 125s 106ms/step - loss: 0.0016 - mae: 0.0239 - val_loss: 0.0075 - val_mae: 0.0517

Epoch 00042: val_loss did not improve from 0.00646
Epoch 43/100
 119/1172 [==>...........................] - ETA: 1:28 - loss: 0.0016 - mae: 0.0236

KeyboardInterrupt: 

## 4. History Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

loss_ax.plot(history.history['loss'], 'y', label='train_loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val_loss')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')

plt.yscale("log")

loss_ax.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

loss_ax.plot(history.history['mae'], 'y', label='train_mae')
loss_ax.plot(history.history['val_mae'], 'r', label='val_mae')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')

plt.yscale("log")

loss_ax.legend()

plt.show()

## 5. Clear models

In [ ]:
from model import remove_models
remove_models()

In [ ]:
model.save('./models/AcademyProject-epoch-001+val_loss-0.0075.hdf5')

In [ ]:
model.optimizer.learning_rate

In [ ]:
# <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=6.25e-05>